In [1]:
!mkdir data release

In [2]:
!wget --no-check-certificate \
  "https://docs.google.com/uc?export=download&id=1ST4C_sqMM5bO-SO4JgWBCbry5XocyrSU" \
  -O ./data/valid.json

--2025-12-12 10:04:31--  https://docs.google.com/uc?export=download&id=1ST4C_sqMM5bO-SO4JgWBCbry5XocyrSU
Resolving docs.google.com (docs.google.com)... 74.125.24.113, 74.125.24.138, 74.125.24.101, ...
Connecting to docs.google.com (docs.google.com)|74.125.24.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1ST4C_sqMM5bO-SO4JgWBCbry5XocyrSU&export=download [following]
--2025-12-12 10:04:32--  https://drive.usercontent.google.com/download?id=1ST4C_sqMM5bO-SO4JgWBCbry5XocyrSU&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 172.253.118.132, 2404:6800:4003:c05::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|172.253.118.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 670 [application/octet-stream]
Saving to: ‘./data/valid.json’

./data/valid.json   100%[===================>]     670  --.-KB/s    in 0s      

Q/A system 코드를 하단에 자유롭게 작성해주세요

TA는 하단 코드 실행 후

./release 폴더 하단에 생성된 파일을 기준으로 평가를 진행합니다
파일명은 {본인학번}.test.json으로 저장

===================== 필수 ===============================

1. ./data/test.json 파일 기준으로 코드 변경
  
  현재는 테스트용 valid.json 이 제공되었지만 test.json 으로 실행 될 수 있도록 코드 수정

2. test.json 형식 확인

  { question : 질문 내용, answer : 정답 }

3. 결과 파일 명은 {학번}.test.json 으로 통일
  
  파일은 ./realease 폴더 아래 위치

  예시) ./release/{학번}.test.json

4. 반드시 실행 여부 테스트 후에 제출, 실행 안 될 시 점수 x  

# 꼭 실행해야 할 코드 위에 (★) 표시

# 0. 드라이브 마운트 (★)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. 필요 라이브러리 설치 및 임포트 (★)

In [4]:
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install -q transformers sentence-transformers gradio chromadb
!pip install -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 10.9 MB/s et

In [5]:
from PIL import Image
import os
import io
import torch
import numpy as np
from transformers import CLIPProcessor, CLIPModel, AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
import gradio as gr
import chromadb
from chromadb.config import Settings
import json

# 2. 텍스트/이미지 임베딩 모델 임포트 (★)

In [6]:
# 텍스트 임베딩 모델
text_embedder =  SentenceTransformer("BAAI/bge-m3")

# 이미지 임베딩 모델
clip_model_name = "openai/clip-vit-base-patch32"
clip_model = CLIPModel.from_pretrained(clip_model_name)
clip_processor = CLIPProcessor.from_pretrained(clip_model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

# 3. ChromaDB 불러오기 (★)

In [7]:
# ChromaDB 불러오기
client = chromadb.PersistentClient(path="drive/MyDrive/Movie_DB")

text_collection = client.get_collection("movie_text_embeddings")
image_collection = client.get_collection("movie_image_embeddings")

# 4. ChromaDB 데이터 search 함수 정의 (★)

In [8]:
def search_with_metadata(query_text=None, query_image=None, k=1):
    if query_image: # query에 이미지가 있는 경우,
        inputs = clip_processor(images=query_image, return_tensors="pt")
        with torch.no_grad():
            q_emb = clip_model.get_image_features(**inputs).cpu().numpy().tolist()

        out = image_collection.query(
            query_embeddings=q_emb,
            n_results=k
        )
    else: # query에 텍스트만 있는 경우
        q_emb = text_embedder.encode(query_text).tolist()

        out = text_collection.query(
            query_embeddings=[q_emb],
            n_results=k
        )

    ids = out["ids"][0]
    metas = out["metadatas"][0]

    return {
        "ids": ids,
        "metadatas": metas,
    }

# 5. Q&A 상호작용을 위한 LLM 모델 로드 (★)

In [9]:
model_name = "Qwen/Qwen2.5-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto",
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

# 6. Q&A LLM 파이프라인 함수 정의 (★)

### 6-1. 질의용 전체 파이프라인 함수 (★)

In [10]:
def answer_query(user_text=None, user_image=None):

    # 1. 검색 결과 top-k row index 가져오기
    match_indices = search_with_metadata(query_text=user_text, query_image=user_image, k=1)

    # 2. 영화 정보 context 구성
    meta = match_indices["metadatas"][0]  # top-1 결과
    title = meta.get("title", "")
    genres = meta.get("genres", "")
    overview = meta.get("overview", "")
    popularity = meta.get("popularity", "")
    release_date = meta.get("release_date", "")
    budget = meta.get("budget", "")
    revenue = meta.get("revenue", "")
    runtime = meta.get("runtime", "")
    context = ""

    context = (
        f"title: {title}\n"
        f"genres: {genres}\n"
        f"overview: {overview}\n"
        f"popularity: {popularity}\n"
        f"release_date: {release_date}\n"
        f"budget: {budget}\n"
        f"revenue: {revenue}\n"
        f"runtime: {runtime}\n"
    )

    # 3. 사용자 입력 텍스트 구성
    user_prompt_text = user_text if user_text else "User upload the image."

    # 4. Qwen2용 messages 구성
    messages = [
        {
            "role": "system",
            "content": (
                "You are a movie expert."
                "Answer the user's questions in natural, smooth english only, using the [reference data] as your knowledge source. "
                "The [reference data] contains movie information you should consult to find the most relevant details for your answer."
                "If the movie the user is asking about does not have sufficiently similar information in the [reference data], using your pre-trained knowledge."
                "Always summarize your answer in a single concise sentence."
            )
        },
        {
            "role": "user",
            "content": (
                f"[Question]\n{user_prompt_text}\n\n"
                f"[reference data]\n{context}\n\n"
                "[Answer]"
            )
        }
    ]

    # 5. apply_chat_template로 모델 입력 생성
    inputs = tokenizer.apply_chat_template(
        messages,
    	add_generation_prompt=True,
    	tokenize=True,
    	return_dict=True,
    	return_tensors="pt",
    ).to(model.device)


    # 6. 생성
    outputs = model.generate(
        **inputs,
        max_new_tokens=64,
    )

    # 7. 모델 출력 디코딩
    generated = tokenizer.decode(
        outputs[0][inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True
    )
    return generated.strip()

### 6-2. json 파일 생성용 추가 파이프라인 (★)

In [11]:
INPUT_DIR = "./data"
OUTPUT_DIR = "./release"

# json 파일 로드용 함수
def load_json(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

# json 파일 저장용 함수
def save_json(path, data):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

########################################################################
FILENAME = "test.json" # 불러올 입력 데이터 파일 이름
FILENAME_SAVE = "202000826.test.json"
########################################################################

input_path = os.path.join(INPUT_DIR, FILENAME)
data = load_json(input_path)

output = [] # 저장할 Q&A 데이터 쌍

for item in data:
  # json 파일에서 질문 가져오기
  question = item['question']
  print(f"▶ Generating answer for: {question}")

  # 각 질문에 대한 대답 생성
  answer = answer_query(question)
  print(f"▶ Generated answer: {answer}")

  # json 파일로 저장하기 위한 포멧 설정
  output_data = {
      "question": question,
      "answer": answer
  }
  output.append(output_data)

output_path = os.path.join(OUTPUT_DIR, FILENAME_SAVE)
save_json(output_path, output)

print(f"✔ Saved: {output_path}")

▶ Generating answer for: List all genres of the movie 'The Shed'.
▶ Generated answer: The movie 'The Shed' is categorized under the genres of Horror and Thriller.
▶ Generating answer for: What is the plot summary of 'Tell Me No Secrets' in one sentence?
▶ Generated answer: Jess Koster, a young attorney in LA, is stalked by the serial killer she is prosecuting in this drama thriller TV movie.
▶ Generating answer for: Which genres does 'Godzilla: The Planet Eater' belong to?
▶ Generated answer: 'Godzilla: The Planet Eater' belongs to the genres of Animation, Science Fiction, Action, and Adventure.
▶ Generating answer for: Summarize the plot of 'Omar Killed Me' in one sentence.
▶ Generated answer: 'Omar Killed Me' is a drama thriller where an innocent Moroccan gardener is convicted of murdering an elderly French woman, sparking a journalist's investigation into the case.
▶ Generating answer for: What is the genre of the movie 'The Nosy One'?
▶ Generated answer: The genre of the movie 'The

#별첨. Gradio 앱 실행

##### 가능한 질의 내용: 제목, 장르, 줄거리, 인기 정도, 출시일, 예산, 수익, 상영시간
### (1) 채팅으로만 질의할 경우
- 텍스트 칸에 영화 관련 질문 작성 후 send

### (2) 이미지(포스터)와 함께 질의할 경우
- 이미지 업로드 후, 텍스트 칸에 질문 작성 후 send
- 예: 포스터 업로드 후, 'Let me know this movie's genres.'


In [12]:
def chat_fn(query, image):
    return answer_query(user_text=query, user_image=image)

demo = gr.Interface(
    fn=chat_fn,
    inputs=[gr.Textbox(label="질문 입력"), gr.Image(type="pil", label="이미지 업로드")],
    outputs=gr.TextArea(label="챗봇 답변", scale=50),
    title="🎬 영화 포스터 Q&A 챗봇",
)

demo.queue(max_size=20)
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8d37eaab7adda1c6b3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# 별첨. ChromaDB 생성 코드

### 1. 학습 데이터 불러오기

In [ ]:
from datasets import load_dataset, Dataset
ds = load_dataset("stzhao/movie_posters_100k_controlnet")

### 2. 학습 데이터 전처리: 텍스트 임베딩용 문자열 생성

In [ ]:
def preprocess(sample):
    # genres 문자열로 변환
    genre_names = [g['name'] for g in sample['genres']]
    genres_str = ", ".join(genre_names)

    # 텍스트 임베딩용 문자열
    sample['text_for_embedding'] = f"{sample['title']}\nGenres: {genres_str}\nOverview: {sample['overview']}\nRelease_date: {sample.get('release_date','')}"
    return sample

dataset = ds.map(preprocess)

### 3. ChromaDB 데이터 생성

In [ ]:
import tqdm

# 1. ChromaDB 초기화
client = chromadb.PersistentClient(path="my_chroma_db")

# 텍스트 임베딩 저장 collection
text_collection = client.get_or_create_collection(
    name="movie_text_embeddings",
    metadata={"description": "영화 텍스트 임베딩 저장"}
)

# 이미지 임베딩 저장 collection
image_collection = client.get_or_create_collection(
    name="movie_image_embeddings",
    metadata={"description": "영화 이미지 임베딩 저장"}
)

# 2. Embedding 생성 및 저장
text_embeddings = []
text_metadatas = []
text_documents = []

image_embeddings = []
image_metadatas = []

id_number = list(map(str, range(1, 91001))) # 총 91000개의 데이터 임베딩용 ID 생성

for sample in tqdm(dataset):

    # 텍스트 임베딩
    t_emb = text_embedder.encode(sample["text_for_embedding"])
    t_emb = t_emb.tolist()

    text_embeddings.append(t_emb)
    text_metadatas.append({
        "title": sample["title"],
        "genres": sample["genres"],
        "overview": sample["overview"],
        "release_date": sample["release_date"],
        "budget": sample["budget"],
        "revenue": sample["revenue"],
        "runtime": sample["runtime"],
    })
    text_documents.append(sample["text_for_embedding"])

    # 이미지 임베딩
    image = sample['image']
    inputs = clip_processor(images=image, return_tensors="pt")

    with torch.no_grad():
        i_emb = clip_model.get_image_features(**inputs)

    i_emb = i_emb.cpu().numpy().squeeze().tolist()

    image_embeddings.append(i_emb)
    image_metadatas.append({
        "title": sample["title"],
        "genres": sample["genres"],
        "overview": sample["overview"],
        "release_date": sample["release_date"],
        "budget": sample["budget"],
        "revenue": sample["revenue"],
        "runtime": sample["runtime"],
    })

### 4. ChromaDB 데이터 저장

In [ ]:
def batched_add(collection, ids, embeddings, documents=None, metadatas=None, batch_size=5000):
    n = len(ids)
    for i in range(0, n, batch_size):
        end = i + batch_size
        batch_ids = ids[i:end]
        batch_emb = embeddings[i:end]
        batch_docs = documents[i:end] if documents is not None else None
        batch_meta = metadatas[i:end] if metadatas is not None else None

        collection.add(
            ids=batch_ids,
            embeddings=batch_emb,
            documents=batch_docs,
            metadatas=batch_meta,
        )

# 메타 데이터를 ChromaDB 형식에 맞게 수정 -----
temp = text_metadatas[:]

for item in temp:
    item['genres'] = ", ".join([g["name"] for g in item['genres']])
# ----------------------------------------------

batched_add(
    text_collection,
    ids=id_number,
    embeddings=text_embeddings,
    documents=text_documents,
    metadatas=temp,
)

batched_add(
    image_collection,
    ids=id_number,
    embeddings=image_embeddings,
    metadatas=temp,
)